In [1]:
%cd ../..

C:\Users\ManuJoseph\OneDrive - Thoucentric\Work\Projects\Playground\AdvancedTimeSeriesForecastingBook\Github\Modern-Time-Series-Forecasting-with-Python-


In [10]:
import numpy as np
import os
import pandas as pd
from pathlib import Path
from tqdm.autonotebook import tqdm
import statsmodels.api as sm
import warnings
import random
from IPython.display import display, HTML
np.random.seed(42)
random.seed(42)
tqdm.pandas()

# Reading and Selecting Households

In [4]:
try:
    lclid_acorn_map = pd.read_pickle("data/london_smart_meters/preprocessed/london_smart_meters_lclid_acorn_map.pkl")
except FileNotFoundError:
    display(HTML("""
    <div class="alert alert-block alert-warning">
    <b>Warning!</b> File not found. Please make sure you have run 02 - Preprocessing London Smart Meter Dataset.ipynb in Chapter02
    </div>
    """))

In [11]:
affluent_households = lclid_acorn_map.loc[lclid_acorn_map.Acorn_grouped=="Affluent", ["LCLid",'file']]
adversity_households = lclid_acorn_map.loc[lclid_acorn_map.Acorn_grouped=="Adversity", ["LCLid",'file']]
comfortable_households = lclid_acorn_map.loc[lclid_acorn_map.Acorn_grouped=="Comfortable", ["LCLid",'file']]


Let's take a subset of the data because if we take everything, it will hit your RAM. Depending on how much RAM you have, we can choose larger data. But to maintain the variety in the dataset, we will do stratified sampling based on Acorn classifications.

* <= 50 households for 4GB RAM
* 50 - 100 households for 8GB RAM
* 100-150 households for 16GB RAM
* 250 households for 32GB RAM

Let's sample 150 households now, but feel free to reduce of increase as per your hardware constraints

150 households means 50 each from the three Acorn Groups - Affluent, Comfortable, Adversity (we are ignoring the households with unknown ACORN groups)

In [41]:
selected_households = pd.concat(
    [
        affluent_households.sample(50, random_state=76),
        comfortable_households.sample(50, random_state=76),
        adversity_households.sample(50, random_state=76),
    ]
)
selected_households['block']=selected_households.file.str.split("_", expand=True).iloc[:,1].astype(int)

In [56]:
# extracting the paths to the different blocks and extracting the starting and ending blocks
path_blocks = [
    (p, *list(map(int, p.name.split("_")[5].split(".")[0].split("-"))))
    for p in Path("data/london_smart_meters/preprocessed").glob(
        "london_smart_meters_merged_block*"
    )
]

In [73]:
household_df_l = []
for path, start_b, end_b in tqdm(path_blocks):
    block_df = pd.read_parquet(path)
    selected_households['block'].between
    mask = selected_households['block'].between(start_b, end_b)
    lclids = selected_households.loc[mask, "LCLid"]
    household_df_l.append(block_df.loc[block_df.LCLid.isin(lclids)])

  0%|          | 0/14 [00:00<?, ?it/s]

In [76]:
block_df = pd.concat(household_df_l)
del household_df_l
block_df.head()

,LCLid,start_timestamp,frequency,energy_consumption,series_length,stdorToU,Acorn,Acorn_grouped,file,holidays,...,windBearing,temperature,dewPoint,pressure,apparentTemperature,windSpeed,precipType,icon,humidity,summary
57,MAC000768,2012-04-21,30min,"[0.8440000000000001, 0.265, 0.262, 0.233999999...",32544,Std,ACORN-A,Affluent,block_1,"[NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDA...",...,"[251, 251, 251, 251, 246, 246, 242, 242, 244, ...","[6.42, 6.42, 6.2, 6.2, 5.68, 5.68, 5.16, 5.16,...","[3.54, 3.54, 3.61, 3.61, 3.52, 3.52, 3.11, 3.1...","[994.96, 994.96, 994.98, 994.98, 994.82, 994.8...","[3.79, 3.79, 3.67, 3.67, 3.15, 3.15, 2.61, 2.6...","[3.64, 3.64, 3.42, 3.42, 3.25, 3.25, 3.13, 3.1...","[rain, rain, rain, rain, rain, rain, rain, rai...","[partly-cloudy-night, partly-cloudy-night, par...","[0.82, 0.82, 0.83, 0.83, 0.86, 0.86, 0.87, 0.8...","[Partly Cloudy, Partly Cloudy, Partly Cloudy, ..."
63,MAC000948,2012-05-02,30min,"[0.008, 0.009, 0.008, 0.008, 0.008, 0.009, 0.0...",32016,Std,ACORN-A,Affluent,block_1,"[NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDA...",...,"[351, 351, 0, 0, 0, 0, 351, 351, 348, 348, 3, ...","[11.81, 11.81, 11.12, 11.12, 11.2, 11.2, 11.18...","[10.47, 10.47, 10.15, 10.15, 9.89, 9.89, 9.29,...","[1021.42, 1021.42, 1021.44, 1021.44, 1021.33, ...","[11.81, 11.81, 11.12, 11.12, 11.2, 11.2, 11.18...","[2.53, 2.53, 2.41, 2.41, 2.06, 2.06, 2.98, 2.9...","[rain, rain, rain, rain, rain, rain, rain, rai...","[partly-cloudy-night, partly-cloudy-night, par...","[0.91, 0.91, 0.94, 0.94, 0.92, 0.92, 0.88, 0.8...","[Mostly Cloudy, Mostly Cloudy, Mostly Cloudy, ..."
2827,MAC003299,2012-09-25,30min,"[0.254, 0.201, 0.183, 0.2189999999999999, 0.18...",25008,Std,ACORN-C,Affluent,block_5,"[NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDA...",...,"[220, 220, 218, 218, 215, 215, 211, 211, 207, ...","[10.93, 10.93, 10.81, 10.81, 10.27, 10.27, 10....","[7.76, 7.76, 8.07, 8.07, 8.04, 8.04, 7.62, 7.6...","[989.26, 989.26, 989.27, 989.27, 989.0, 989.0,...","[10.93, 10.93, 10.81, 10.81, 10.27, 10.27, 10....","[4.9, 4.9, 4.98, 4.98, 4.45, 4.45, 4.51, 4.51,...","[rain, rain, rain, rain, rain, rain, rain, rai...","[clear-night, clear-night, clear-night, clear-...","[0.81, 0.81, 0.83, 0.83, 0.86, 0.86, 0.84, 0.8...","[Clear, Clear, Clear, Clear, Clear, Clear, Cle..."
3389,MAC003157,2012-07-15,30min,"[0.181, 0.126, 0.13, 0.134, 0.18, 0.179, 0.118...",28464,ToU,ACORN-C,Affluent,block_6,"[NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDA...",...,"[250, 250, 259, 259, 267, 267, 284, 284, 286, ...","[12.72, 12.72, 12.72, 12.72, 12.98, 12.98, 12....","[12.01, 12.01, 12.0, 12.0, 12.32, 12.32, 11.83...","[1011.2, 1011.2, 1011.17, 1011.17, 1011.23, 10...","[12.72, 12.72, 12.72, 12.72, 12.98, 12.98, 12....","[1.73, 1.73, 2.15, 2.15, 2.31, 2.31, 2.28, 2.2...","[rain, rain, rain, rain, rain, rain, rain, rai...","[clear-night, clear-night, partly-cloudy-night...","[0.95, 0.95, 0.95, 0.95, 0.96, 0.96, 0.95, 0.9...","[Clear, Clear, Partly Cloudy, Partly Cloudy, P..."
3916,MAC000193,2012-01-01,30min,"[0.368, 0.386, 0.17, 0.021, 0.038, 0.038, 0.02...",37872,ToU,ACORN-D,Affluent,block_7,"[NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDAY, NO_HOLIDA...",...,"[229, 229, 238, 238, 229, 229, 231, 231, 227, ...","[12.12, 12.12, 12.59, 12.59, 12.45, 12.45, 12....","[10.97, 10.97, 11.02, 11.02, 11.04, 11.04, 10....","[1008.1, 1008.1, 1007.88, 1007.88, 1007.95, 10...","[12.12, 12.12, 12.59, 12.59, 12.45, 12.45, 12....","[5.9, 5.9, 6.06, 6.06, 5.31, 5.31, 4.68, 4.68,...","[rain, rain, rain, rain, rain, rain, rain, rai...","[partly-cloudy-night, partly-cloudy-night, clo...","[0.93, 0.93, 0.9, 0.9, 0.91, 0.91, 0.93, 0.93,...","[Mostly Cloudy, Mostly Cloudy, Overcast, Overc..."


In [77]:
from src.utils.data_utils import compact_to_expanded

In [78]:
#Converting to expanded form
exp_block_df = compact_to_expanded(block_df, timeseries_col = 'energy_consumption',
static_cols = ["frequency", "series_length", "stdorToU", "Acorn", "Acorn_grouped", "file"],
time_varying_cols = ['holidays', 'visibility', 'windBearing', 'temperature', 'dewPoint',
       'pressure', 'apparentTemperature', 'windSpeed', 'precipType', 'icon',
       'humidity', 'summary'],
ts_identifier = "LCLid")

exp_block_df.head()

  0%|          | 0/150 [00:00<?, ?it/s]

,timestamp,LCLid,energy_consumption,frequency,series_length,stdorToU,Acorn,Acorn_grouped,file,holidays,...,windBearing,temperature,dewPoint,pressure,apparentTemperature,windSpeed,precipType,icon,humidity,summary
0,2012-04-21 00:00:00,MAC000768,0.844,30min,32544,Std,ACORN-A,Affluent,block_1,NO_HOLIDAY,...,251,6.42,3.54,994.96,3.79,3.64,rain,partly-cloudy-night,0.82,Partly Cloudy
1,2012-04-21 00:30:00,MAC000768,0.265,30min,32544,Std,ACORN-A,Affluent,block_1,NO_HOLIDAY,...,251,6.42,3.54,994.96,3.79,3.64,rain,partly-cloudy-night,0.82,Partly Cloudy
2,2012-04-21 01:00:00,MAC000768,0.262,30min,32544,Std,ACORN-A,Affluent,block_1,NO_HOLIDAY,...,251,6.20,3.61,994.98,3.67,3.42,rain,partly-cloudy-night,0.83,Partly Cloudy
3,2012-04-21 01:30:00,MAC000768,0.234,30min,32544,Std,ACORN-A,Affluent,block_1,NO_HOLIDAY,...,251,6.20,3.61,994.98,3.67,3.42,rain,partly-cloudy-night,0.83,Partly Cloudy
4,2012-04-21 02:00:00,MAC000768,0.046,30min,32544,Std,ACORN-A,Affluent,block_1,NO_HOLIDAY,...,246,5.68,3.52,994.82,3.15,3.25,rain,clear-night,0.86,Clear


## Reduce Memory Footprint

In [79]:
from src.utils.data_utils import reduce_memory_footprint

In [80]:
exp_block_df.info(memory_usage="deep", verbose=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4711440 entries, 0 to 33263
Columns: 21 entries, timestamp to summary
dtypes: datetime64[ns](1), float64(8), int64(2), object(10)
memory usage: 3.3 GB


In [81]:
exp_block_df = reduce_memory_footprint(exp_block_df)

In [82]:
exp_block_df.info(memory_usage="deep", verbose=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4711440 entries, 0 to 33263
Columns: 21 entries, timestamp to summary
dtypes: category(10), datetime64[ns](1), float32(8), int32(2)
memory usage: 301.1 MB


# Train Test Valildation Split

We are going to keep 2014 data as the validation and test period. We have 2 months(Jan and Feb) of data in 2014. Jan is Validation and Feb is Test

In [83]:
test_mask = (exp_block_df.timestamp.dt.year==2014) & (exp_block_df.timestamp.dt.month==2)
val_mask = (exp_block_df.timestamp.dt.year==2014) & (exp_block_df.timestamp.dt.month==1)

train = exp_block_df[~(val_mask|test_mask)]
val = exp_block_df[val_mask]
test = exp_block_df[test_mask]
print(f"# of Training samples: {len(train)} | # of Validation samples: {len(val)} | # of Test samples: {len(test)}")
print(f"Max Date in Train: {train.timestamp.max()} | Min Date in Validation: {val.timestamp.min()} | Min Date in Test: {test.timestamp.min()}")

# of Training samples: 4293840 | # of Validation samples: 223200 | # of Test samples: 194400
Max Date in Train: 2013-12-31 23:30:00 | Min Date in Validation: 2014-01-01 00:00:00 | Min Date in Test: 2014-02-01 00:00:00


In [84]:
train.to_parquet("data/london_smart_meters/preprocessed/selected_blocks_train.parquet")
val.to_parquet("data/london_smart_meters/preprocessed/selected_blocks_val.parquet")
test.to_parquet("data/london_smart_meters/preprocessed/selected_blocks_test.parquet")

## Train Test Split after filling in missing values

In [85]:
from src.imputation.interpolation import SeasonalInterpolation

block_df.energy_consumption = block_df.energy_consumption.progress_apply(lambda x: SeasonalInterpolation(seasonal_period=48*7).fit_transform(x.reshape(-1,1)).squeeze())

  0%|          | 0/150 [00:00<?, ?it/s]

In [86]:
#Converting to expanded form
exp_block_df = compact_to_expanded(block_df, timeseries_col = 'energy_consumption',
static_cols = ["frequency", "series_length", "stdorToU", "Acorn", "Acorn_grouped", "file"],
time_varying_cols = ['holidays', 'visibility', 'windBearing', 'temperature', 'dewPoint',
       'pressure', 'apparentTemperature', 'windSpeed', 'precipType', 'icon',
       'humidity', 'summary'],
ts_identifier = "LCLid")

exp_block_df.head()

  0%|          | 0/150 [00:00<?, ?it/s]

,timestamp,LCLid,energy_consumption,frequency,series_length,stdorToU,Acorn,Acorn_grouped,file,holidays,...,windBearing,temperature,dewPoint,pressure,apparentTemperature,windSpeed,precipType,icon,humidity,summary
0,2012-04-21 00:00:00,MAC000768,0.844,30min,32544,Std,ACORN-A,Affluent,block_1,NO_HOLIDAY,...,251,6.42,3.54,994.96,3.79,3.64,rain,partly-cloudy-night,0.82,Partly Cloudy
1,2012-04-21 00:30:00,MAC000768,0.265,30min,32544,Std,ACORN-A,Affluent,block_1,NO_HOLIDAY,...,251,6.42,3.54,994.96,3.79,3.64,rain,partly-cloudy-night,0.82,Partly Cloudy
2,2012-04-21 01:00:00,MAC000768,0.262,30min,32544,Std,ACORN-A,Affluent,block_1,NO_HOLIDAY,...,251,6.20,3.61,994.98,3.67,3.42,rain,partly-cloudy-night,0.83,Partly Cloudy
3,2012-04-21 01:30:00,MAC000768,0.234,30min,32544,Std,ACORN-A,Affluent,block_1,NO_HOLIDAY,...,251,6.20,3.61,994.98,3.67,3.42,rain,partly-cloudy-night,0.83,Partly Cloudy
4,2012-04-21 02:00:00,MAC000768,0.046,30min,32544,Std,ACORN-A,Affluent,block_1,NO_HOLIDAY,...,246,5.68,3.52,994.82,3.15,3.25,rain,clear-night,0.86,Clear


## Reduce Memory Footprint

In [87]:
from src.utils.data_utils import reduce_memory_footprint

In [88]:
exp_block_df.info(memory_usage="deep", verbose=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4711440 entries, 0 to 33263
Columns: 21 entries, timestamp to summary
dtypes: datetime64[ns](1), float64(8), int64(2), object(10)
memory usage: 3.3 GB


In [89]:
exp_block_df = reduce_memory_footprint(exp_block_df)

In [90]:
exp_block_df.info(memory_usage="deep", verbose=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4711440 entries, 0 to 33263
Columns: 21 entries, timestamp to summary
dtypes: category(10), datetime64[ns](1), float32(8), int32(2)
memory usage: 301.1 MB


In [91]:
test_mask = (exp_block_df.timestamp.dt.year==2014) & (exp_block_df.timestamp.dt.month==2)
val_mask = (exp_block_df.timestamp.dt.year==2014) & (exp_block_df.timestamp.dt.month==1)

train = exp_block_df[~(val_mask|test_mask)]
val = exp_block_df[val_mask]
test = exp_block_df[test_mask]
print(f"# of Training samples: {len(train)} | # of Validation samples: {len(val)} | # of Test samples: {len(test)}")
print(f"Max Date in Train: {train.timestamp.max()} | Min Date in Validation: {val.timestamp.min()} | Min Date in Test: {test.timestamp.min()}")

# of Training samples: 4293840 | # of Validation samples: 223200 | # of Test samples: 194400
Max Date in Train: 2013-12-31 23:30:00 | Min Date in Validation: 2014-01-01 00:00:00 | Min Date in Test: 2014-02-01 00:00:00


In [92]:
train.to_parquet("data/london_smart_meters/preprocessed/selected_blocks_train_missing_imputed.parquet")
val.to_parquet("data/london_smart_meters/preprocessed/selected_blocks_val_missing_imputed.parquet")
test.to_parquet("data/london_smart_meters/preprocessed/selected_blocks_test_missing_imputed.parquet")